In [1]:
from experiment import Experiment
from utils.behavior_data import BehaviorData
from visuals import Plotter
import torch
import numpy as np
from utils.state_data import StateData
import matplotlib.pyplot as plt

In [ ]:
model, learning_rate = "AdaptableLSTM", .025
# model, learning_rate = "BasicNN", .0024
# model, learning_rate = "LogisticRegressor", .003
epochs = 1000
seed = 1
include_state = True
estate = True
fullq = True
respond_perc = .50
fullseq = False
insertpreds = True
noise = .05
smooth = .1

    
np.random.seed(seed)
torch.manual_seed(seed)


e = Experiment(
    numValFolds = 5,
    epochsToUpdateLabelMods = 5,
    data_kw={"minw": 2,
            "maxw": 29,
            "include_state": include_state,
            "include_pid": False,
            "expanded_states": estate,
            "top_respond_perc": respond_perc,
             "full_questionnaire": fullq,
             "full_sequence": fullseq,
             "insert_predictions": insertpreds,
             "one_hot_response_features": True,
             "response_feature_noise": noise
            },
    model=model,
    model_kw={
        "lossfn": "MSELoss",
        # "lossfn": "NDCG",
        # "lossfn": "CrossEntropyLoss",
        "hidden_size": 200,
        "lr_step_mult": .9, 
        "lr_step_epochs": 100,
        "opt_kw": {
            "lr": learning_rate,
        },
        "labelSmoothPerc": smooth,
        "gaussianNoiseStd": noise
        
    },
    train_kw={
        "epochs": epochs,
        "n_subj": 500,
        "rec_every": 5,
    })


print(len(e.bd.test))
print(len(e.bd.train))

report = e.run()






268 537
(6359, 73) (6359, 8) 73
54
214
0	 train loss: 0.1521 train acc: 51.598% test acc: 51.149%
Some problem tensor([0.2775, 0.3557, 0.3668])
Some problem tensor([0.3972, 0.1348, 0.4679])
Some problem tensor([0.2702, 0.3664, 0.3635])
Some problem tensor([0.3965, 0.1356, 0.4679])
Some problem tensor([0.3934, 0.1352, 0.4714])
Some problem tensor([0.2779, 0.3556, 0.3665])
Some problem tensor([0.2795, 0.3495, 0.3710])
Some problem tensor([0.2716, 0.3605, 0.3680])
Some problem tensor([0.3997, 0.1342, 0.4660])
Some problem tensor([0.2723, 0.3643, 0.3635])
Some problem tensor([0.2694, 0.3647, 0.3659])
Some problem tensor([0.3977, 0.1332, 0.4691])
Some problem tensor([0.2724, 0.3614, 0.3662])
Some problem tensor([0.3975, 0.1358, 0.4668])
Some problem tensor([0.3969, 0.1363, 0.4668])
Some problem tensor([0.3837, 0.1738, 0.4425])
Some problem tensor([0.2783, 0.3505, 0.3711])
Some problem tensor([0.2709, 0.3628, 0.3663])
Some problem tensor([0.3988, 0.1332, 0.4681])
Some problem tensor([0.2693,

In [ ]:

print (np.mean(report['train_metrics'], axis=0))
labels = report["metric_labels"]
print(report['train_metrics'][-1, labels.index("Acc")])
print(report['test_metrics'][-1, labels.index("Acc")])

splot = plt.plot(report["rec_epochs"], report["train_metrics"][:, labels.index("Acc")], label="Train Acc.")
splot = plt.plot(report["rec_epochs"], report["test_metrics"][:, labels.index("Acc")], label="Test Acc.")
splot = plt.plot(report["rec_epochs"], report["train_metrics"][:, labels.index("MSE")], label="Train MSE")
splot = plt.plot(report["rec_epochs"], report["test_metrics"][:, labels.index("MSE")], label="Test MSE")
plt.title("Train/Test Performance Over Training")
plt.legend()
plt.ylabel("Metric")
plt.xlabel("Training Epoch")
plt.savefig("simpleNotebookAccPlot.png")

plt.clf()

In [ ]:
print(e.bd.features.shape, e.bd.featureList.shape)